This notebook is meant to be used to register copernicus data in csv files.

Data source : https://cds.climate.copernicus.eu/cdsapp#!/dataset/projections-cmip6?tab=form

# User input

In [1]:
# CAREFULE : NEED TO CHECK IF WORKED FOR FUTURE PRECIPITATION, NEED TO CHECK TEMPERATURE NEED COnversion
period_of_interest = 'past' # 'past' or 'future'
name_variable = 'surface_downwelling_longwave_radiation'
global_variable = 'radiation'
name_dataset_variable = 'rlds' # name of the column of interest

# 'tas' 'near_surface_air_temperature'  --> past done
# 'tasmax' 'daily_maximum_near_surface_air_temperature' --> done
# 'tasmin' 'daily_minimum_near_surface_air_temperature' --> done
# 'huss' 'near_surface_specific_humidity' --> past,future done
# 'hurs' 'near_surface_relative_humidity', 'monthly'
# 'rlds 'radiation longwave 'surface_downwelling_longwave_radiation' 'monthly'
# 'rsds' radiation shortwave 'surface_downwelling_shortwave_radiation' 'monthly'
# 'Wind' 'near_surface_wind_speed' --> done

temporal_resolution = 'monthly'

# wind registered at 10 m

# Functions and packages

In [2]:
import pandas as pd
import numpy as np
import numpy.ma as ma
import netCDF4 as nc#not directly used but needs to be imported for some nc4 files manipulations, use for nc files
from netCDF4 import Dataset
import xarray as xr
import datetime # to have actual date
import os
import os.path
import cdsapi # for copernicus function
import shutil
from datetime import datetime

# Out path

In [3]:
out_path=r'\\COWI.net\projects\A245000\A248363\CRVA\Datasets'

# Project information

In [4]:
name_projects = np.array(['WTP_Mutua_EIB', 'Gorongosa_EIB', 'Chimoio_WTP_EIB', 'Pemba_EIB'])

lon_projects_data = np.array([34.5927839939706, 34.07824286310398 , 33.47333313659342, 40.52545156033736])
lon_projects = pd.Series(lon_projects_data)

lat_projects_data = np.array([-19.495079648575242, -18.68063728746643, -19.125095255188334,-12.973942656747809])
lat_projects = pd.Series(lat_projects_data)
buffer_area_project=2
area_projects = [lat_projects - buffer_area_project, lat_projects+buffer_area_project, lon_projects-buffer_area_project,lon_projects+buffer_area_project] # list format

In [5]:
def check_validity_name(name_projects):
    for name_project in name_projects:
        # modification on name_project str to ensure no problem whent using this str as name of a folder
        name_project = name_project.replace('-','_') # take off every blank space of project names
        name_project = name_project.replace('/','_') # take off every / of project names
        name_project = name_project.replace(r'"\"','_') # take off every \ of project names
        # brackets shouldn't be a problem for name projects'''
    return name_projects

In [6]:
name_projects=check_validity_name(name_projects)

# Class

In [7]:
class calendar:
    default_month = [ 
                '01', '02', '03',
                '04', '05', '06',
                '07', '08', '09',
                '10', '11', '12',
                ]
    default_day = [
                '01', '02', '03',
                '04', '05', '06',
                '07', '08', '09',
                '10', '11', '12',
                '13', '14', '15',
                '16', '17', '18',
                '19', '20', '21',
                '22', '23', '24',
                '25', '26', '27',
                '28', '29', '30',
                '31',
                ]

In [8]:
## Definition of tuples that will be useful to search which data are available or not
# make it tuples to make unchangeable
class copernicus_elements:
    # there is 58 models
    models =('access_cm2','awi_cm_1_1_mr','bcc_csm2_mr','cams_csm1_0','canesm5_canoe','cesm2_fv2','cesm2_waccm_fv2','cmcc_cm2_hr4','cmcc_esm2','cnrm_cm6_1_hr','e3sm_1_0','e3sm_1_1_eca','ec_earth3_aerchem','ec_earth3_veg','fgoals_f3_l','fio_esm_2_0','giss_e2_1_g','hadgem3_gc31_ll','iitm_esm','inm_cm5_0','ipsl_cm6a_lr','kiost_esm','miroc6','miroc_es2l','mpi_esm1_2_hr','mri_esm2_0','norcpm1','noresm2_mm','taiesm1','access_esm1_5','awi_esm_1_1_lr','bcc_esm1','canesm5','cesm2','cesm2_waccm','ciesm','cmcc_cm2_sr5','cnrm_cm6_1','cnrm_esm2_1','e3sm_1_1','ec_earth3','ec_earth3_cc','ec_earth3_veg_lr','fgoals_g3','gfdl_esm4','giss_e2_1_h','hadgem3_gc31_mm','inm_cm4_8','ipsl_cm5a2_inca','kace_1_0_g','mcm_ua_1_0','miroc_es2h','mpi_esm_1_2_ham','mpi_esm1_2_lr','nesm3','noresm2_lm','sam0_unicon','ukesm1_0_ll')
    experiments = ('ssp1_1_9','ssp1_2_6','ssp4_3_4','ssp5_3_4os','ssp2_4_5','ssp4_6_0','ssp3_7_0','ssp5_8_5')
    experiments_historical=('historical',)

In [9]:
if period_of_interest == 'past':
    scenarios=copernicus_elements.experiments_historical
    y_start = 1950
    y_end = 2014
if period_of_interest == 'future':
    scenarios=copernicus_elements.experiments
    y_start = 2015
    y_end = 2100

# Functions

In [10]:
########################################### Register data from nc file of Copernicus ############################################
# Aim of the function: this function aims to register in a dataframe and a csv file the data from the nc file downloaded with
# the function copernicus_data
# Actions of this function
#     1) Create the string indicating the period of interest
#     2) Creating path and file name to register dataframe in csv file
#     3) Register data, with its corresponding experiments and models, in dataframe and csv file
#        3 a) Test if path does not exists (if dataframe is not registered) : 
#                1 . Thanks to copernicus_data, download nc fils from copernicus CMIP6 website for each experiment and each model
#                2 . Open the dowloaded nc file in the jupyter notebook if it exists
#                3 . In a dataframe, register the value in the nc file, for each experiment, model and day
#                4 . If there no value for each experiments and models tested, the datfram is empty and the user is informed
#        3 b) Test if path exists (dataframe is registered) : no need to register again, return in dataframe the existing 
#             csv file in a dataframe

# Parameters of the function
# temporal_resolution: 'daily', 'monthly', or 'fixed'. String type 
# year_str: list containing all the years under the string type and in the period of interest
# experiments: copernicus_elements.experiments
# models: copernicus_elements.models
# out_path: path were the outputs are registered. Defined by the user at the beginning of the code 
# name_dataset_variable: name of the column of interest in the dataset
# name_variable: name of the elements downloaded from copernicus (example: 'near_surface_wind_speed')
# global_variable: for exemaple, Wind, or temperature
# name_project: Name of the project for which the data are taken
# area: list containing latitudes and logitudes around the project

def csv_copernicus(temporal_resolution,year_str,experiments,models,out_path, name_dataset_variable, name_variable,global_variable, name_projects,area,lat_projects,lon_projects,source = 'Copernicus-CMIP6',name_area='all-Mozambique'):    
    df = pd.DataFrame()
    
    # create string for name of folder depending on type of period
    if temporal_resolution == 'fixed':
        period = 'fixed'
    else:
        period=year_str[0]+'-'+year_str[len(year_str)-1]
    
    conversion_factor = 1
    conversion_addition = 0
    if name_dataset_variable =='pr':
        conversion_factor = 86400
        unit = 'mm_per_day'
        # convert precipitation data from kg.m^(-2).s^(-1) to mm/day :  1 kg/m2/s = 86400 mm/day
    if 'temp' in name_variable.lower():
        unit = u'\N{DEGREE SIGN}C'
        conversion_addition = -273.15
    if 'wind' in name_variable.lower():
        unit = 'm_per_s'
    if name_dataset_variable =='huss':
        unit = '[-]' # dimensionless
    if name_dataset_variable =='huRs':
        unit = '%'
    if 'radiation' in name_variable.lower():
        unit ='MJ_per_m-2_per_day-1'
        conversion_factor=8.64 * 10**(-2)
        # original units of rsds is W m-2, but need them in MJ.m-2.day-1. The conversion factor is 8.64 * 10**(-2)
    title_file = 'SomeProjectsInMoz_'+period+ '_' + temporal_resolution + '_' +name_variable+'_'+unit+'.csv'

    path_for_csv = os.path.join(out_path,global_variable,name_variable,source,'csv',period) # create path for csv file
    
    if os.path.isfile(os.path.join(path_for_csv,title_file)):
        df = file_already_downloaded(path_for_csv,title_file)
        return df
    else:
        for SSP in os.listdir(os.path.join(out_path,global_variable,name_variable,source,'raw_data',period)):
            for model_simulation in os.listdir(os.path.join(out_path,global_variable,name_variable,source,'raw_data',period,SSP)):
                path_for_file = os.path.join(out_path,global_variable,name_variable,source,'raw_data',period,SSP,model_simulation,name_area)
                path_nc_file = search_for_nc(path_for_file)
                
                if path_nc_file!=[]:
                    (index_closest_lat,index_closest_lon,closest_value_lat,closest_value_lon) = _lat_lon(path_nc_file,lat_projects,lon_projects)

                    ds = xr.open_dataset(path_nc_file)

                    for (name_project,i) in zip(name_projects,np.arange(0,len(name_projects))):
                        data_dataframe = ds.variables[name_dataset_variable].isel(lat=index_closest_lat[i],lon=index_closest_lon[i]).values*conversion_factor+conversion_addition

                        # prevent problemes with dates
                        # 2 types of problems :
                        #     dates and time in ds are not overlaping
                        #     missing 29.02 in ds
                        if type(ds.indexes['time']) == pd.core.indexes.datetimes.DatetimeIndex:
                            dates=ds.indexes['time'].floor('D')
                        if type(ds.indexes['time']) == xr.coding.cftimeindex.CFTimeIndex:
                            dates=ds.indexes['time'].to_datetimeindex().floor('D')
                        else:
                            type_ = type(ds.indexes['time'])
                            print(f'The type of the time index is {type_}')

                        Date = dates.tolist() # create tuple for iteration of dataframe

                        Name_Project = (name_project,)

                        # Create the MultiIndex
                        midx = pd.MultiIndex.from_product([Name_Project,(closest_value_lat[i],),(closest_value_lon[i],),(SSP,), (model_simulation,), Date],names=['Name project', 'Latitude', 'Longitude','Experiment', 'Model', 'Date'])
                        # multiindex to name the columns
                        cols_str = [name_variable+' '+unit]
                        #cols = pd.MultiIndex.from_product([lon_str,lon_dataframe])
                        # Create the Dataframe
                        Variable_dataframe = pd.DataFrame(data = data_dataframe, 
                                                    index = midx,
                                                    columns = cols_str)
                        # Concatenate former and new dataframe
                        df = pd.concat([df,Variable_dataframe])# register information for project

                    ds.close() # to spare memory
            
    # register in csv
    full_path = os.path.join(path_for_csv,title_file)
    if not os.path.isfile(path_for_csv):
        os.makedirs(path_for_csv)
    df.to_csv(full_path)
    print(f'csv file registered here : {full_path}')
    return df#df_final

In [11]:
def file_already_downloaded(path_for_csv,title_file):
    print(f'The file {title_file} exists and is in {path_for_csv}')
    df = pd.read_csv(os.path.join(path_for_csv,title_file)) # read the downloaded data for the analysis

    df.set_index(['Name project', 'Latitude', 'Longitude','Experiment', 'Model', 'Date'],inplace=True)

    return df

In [12]:
# seach_for_nc is a function looking in path_for_file for a document in .nc format

def search_for_nc(path_for_file):
    print('path_for_file '+path_for_file+' does exist Function copernicus search for nc')
    for file in os.listdir(path_for_file):
        if file.endswith(".nc"):
            final_path=os.path.join(path_for_file, file)
            
            print('The file is in the path Function copernicus search for nc\n')
            print('Before path_length, The final path for the nc file is: '+final_path)
            answer = str(os.path.isfile(final_path))
            print('\n The final path for nc file exists ? '+answer+'\n')
            final_path=path_length(final_path) # check if length of path is too long
            print('After path_length, The final path for the nc file is: '+final_path)
            answer = str(os.path.isfile(final_path))
            print('\n The final path for nc file exists ? '+answer+'\n')
            return final_path # the function returns the path of the nc file of interest
            break # stop the function if a nc file was found 
        else:
            pass
    # the all folder has been search and there is no nc file in it
    print('Problem : No nc file was found Function copernicus Function copernicus search for nc')# this line is out of the for loop, 
    return []
    #because it should only appear once all the folder has been examinated and if the break of the if was not used

In [13]:
# this functions test if the path is too long
# if the path is more than 250 char, the path wll be modified in order for windows to accept is as a path

def path_length(str1):
    if len(str1)>250:
        path = os.path.abspath(str1) # normalize path
        if path.startswith(u"\\\\"):
            path=u"\\\\?\\UNC\\"+path[2:]
        else:
            path=u"\\\\?\\"+path
        return path
    else:
        return str1

In [14]:
# this functions aims to return the closest latitudes and longitudes to the projects, and the respectives index 
#  in the lat and lon vectors of the file
def _lat_lon(path,lat_projects,lon_projects):
    ds =  xr.open_dataset(path) 
    # ds.indexes['time'] gives back CFTimeIndex format, with hours. The strftime('%d-%m-%Y') permits to have time 
    # as an index, with format '%d-%m-%Y'. The .values permits to have an array
    lat  = ds.lat.values
    lon  = ds.lon.values
    ds.close() # to spare memory
    # preallocate space for the future vectors
    index_closest_lat = []
    index_closest_lon = []
    closest_value_lat = []
    closest_value_lon = []
    for j in np.arange(0,len(lat_projects)):
        (A,B)=closest_lat_lon_to_proj(lat_projects[j],lat)
        #return lat,lat_projects[j]
        index_closest_lat.append(A[0])
        closest_value_lat.append(B[0])
        (C,D)=closest_lat_lon_to_proj(lon_projects[j],lon)
        index_closest_lon.append(C[0])
        closest_value_lon.append(D[0])
    return index_closest_lat,index_closest_lon,closest_value_lat,closest_value_lon


# this function aims to select the closest point to the geographical point of the project
# the function takes as input 
#     location_project, which is a numpy.float64
#     vector, which is a numpy.ndarray
# the function returns
#     closest_value[0], a numpy.float64

def closest_lat_lon_to_proj(location_project,vector):
    # the function any() returns a boolean value. Here, the function test if there are elements in the array 
    # containing the difference between the vector and the location_project, equal to the minimum of the absolute 
    # value of the difference between the vector and the location_project
    if any(np.where((vector - location_project) == min(abs(vector - location_project)))[0]):
        # the function any() returned True
        # there is an element in the vector that is equal to the minimum of the absolute value of the difference 
        # between the vector and the location_project
        
        # the function np.where() returns the index for which (vector - location_project) == min(abs(vector - location_project))
        index_closest = np.where((vector - location_project) == min(abs(vector - location_project)))[0]
        closest_value = vector[index_closest]
    else:
        # the function any() returned False
        # there is NO element in the vector that is equal to the minimum of the absolute value of the difference 
        # between the vector and the location_project
        
        # the function np.where() returns the index for which (vector - location_project) == -min(abs(vector - location_project))
        index_closest = np.where((vector - location_project) == -min(abs(vector - location_project)))[0]
        closest_value = vector[index_closest]
    return index_closest, closest_value 
    # the function returns
    #     first, the value of the index of the element of vector, that is the closest to location_project    
    #     second, the array containing the element of vector, that is the closest to location_project


In [15]:
################################################ Period for copernicus function ################################################
# Aim of the function: by giving it a first and last year of the period that must analyzed, this function produce several 
# vectors,containing time informations, useful to download and treat data from CMIP6 projections (https://cds.climate.copernicus.eu/cdsapp#!/dataset/projections-cmip6?tab=overview )
# Those time vectors are used in the copernicus_data and the dataframe_copernicus and csv_copernicus functions

# function year_copernicus produce 
# year: a vector containing all the year in the period of interest
# year_str: an array containing all the year in the period of interest in the string format
# index: an array containing the index of the year and year_str
#### Parameters of the function
# first_year: number in int format, of the first year of the period of interest
# last_year: number in int format, of the last year of the period of interest
def year_copernicus(first_year,last_year):
    year = np.arange(first_year,(last_year+1),1) # create vector of years
    year_str = [0]*len(year) # create initiale empty vector to convert years in int
    index = np.arange(0,len(year)) # create vector of index for year
    i = 0 # initialize index
    for i in index: # convert all the date in string format
        year_str[i]=str(year[i])
    return (year, year_str, index)

# function date_copernicus produce 
# dates: the format depend on the temporal reolution, but always contain the dates of the period of interest.
#        with temporal_resolution=daily, dates is a DatetimeIndex
#        with temporal_resolution=monthly, dates is a list
# index_dates: an array containing the index of the dates
#### Parameters of the function
# temporal_resolution: daily or monthly
# year_str: ???? produce by function year_copernicus, containing the year of the period of interest in string format
def date_copernicus(temporal_resolution,year_str):
    start_date = "01-01-"+year_str[0] # string start date based on start year
    stop_date = "31-12-"+year_str[len(year_str)-1] # string stop date based on stop year
    if temporal_resolution =='daily':
        # vector of dates between start date and stop date
        dates = pd.date_range(start_date,stop_date)# dates is a pandas.core.indexes.datetimes.DatetimeIndex
        # By default, freq = 'D', which means calendar day frequency (source : https://pandas.pydata.org/docs/user_guide/timeseries.html#timeseries-offset-aliases)
        #index_dates = np.arange(0,len(dates)) # vector containning index o dates vector
    if temporal_resolution =='monthly':
        dates = pd.date_range(start_date,stop_date,freq='MS') # vector of dates between start date and stop date
        dates=list(dates.strftime('%m-%Y')) # dates is an pandas.core.indexes.base.Index, not a pandas.core.indexes.datetimes.DatetimeIndex
    #if temporal_resolution =='fixed': trouver donnees pour gerer cela
    index_dates = np.arange(0,len(dates)) # vector containning index o dates vector
    return (dates, index_dates)

In [16]:
(year, year_str, index)=year_copernicus(y_start,y_end)

In [17]:
df=csv_copernicus(temporal_resolution,year_str,scenarios,copernicus_elements.models,out_path, name_dataset_variable, name_variable,global_variable, name_projects,area_projects,lat_projects,lon_projects)

path_for_file \\COWI.net\projects\A245000\A248363\CRVA\Datasets\radiation\surface_downwelling_longwave_radiation\Copernicus-CMIP6\raw_data\1950-2014\historical\cesm2\all-Mozambique does exist Function copernicus search for nc
The file is in the path Function copernicus search for nc

Before path_length, The final path for the nc file is: \\COWI.net\projects\A245000\A248363\CRVA\Datasets\radiation\surface_downwelling_longwave_radiation\Copernicus-CMIP6\raw_data\1950-2014\historical\cesm2\all-Mozambique\rlds_Amon_CESM2_historical_r1i1p1f1_gn_19500115-20141215_v20190308.nc

 The final path for nc file exists ? True

After path_length, The final path for the nc file is: \\COWI.net\projects\A245000\A248363\CRVA\Datasets\radiation\surface_downwelling_longwave_radiation\Copernicus-CMIP6\raw_data\1950-2014\historical\cesm2\all-Mozambique\rlds_Amon_CESM2_historical_r1i1p1f1_gn_19500115-20141215_v20190308.nc

 The final path for nc file exists ? True



C:\Users\CLMRX\AppData\Local\miniconda3\envs\geodata\lib\site-packages\xarray\backends\plugins.py:80: RuntimeWarning: Engine 'cfgrib' loading failed:
Cannot find the ecCodes library
  warnings.warn(f"Engine {name!r} loading failed:\n{ex}", RuntimeWarning)
C:\Users\CLMRX\AppData\Local\Temp\1\ipykernel_916\3903570655.py:84: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  dates=ds.indexes['time'].to_datetimeindex().floor('D')
C:\Users\CLMRX\AppData\Local\Temp\1\ipykernel_916\3903570655.py:84: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  dates=ds.indexes['time'].to_dat

path_for_file \\COWI.net\projects\A245000\A248363\CRVA\Datasets\radiation\surface_downwelling_longwave_radiation\Copernicus-CMIP6\raw_data\1950-2014\historical\cesm2_fv2\all-Mozambique does exist Function copernicus search for nc
The file is in the path Function copernicus search for nc

Before path_length, The final path for the nc file is: \\COWI.net\projects\A245000\A248363\CRVA\Datasets\radiation\surface_downwelling_longwave_radiation\Copernicus-CMIP6\raw_data\1950-2014\historical\cesm2_fv2\all-Mozambique\rlds_Amon_CESM2-FV2_historical_r1i1p1f1_gn_19500115-20141215_v20191120.nc

 The final path for nc file exists ? True

After path_length, The final path for the nc file is: \\COWI.net\projects\A245000\A248363\CRVA\Datasets\radiation\surface_downwelling_longwave_radiation\Copernicus-CMIP6\raw_data\1950-2014\historical\cesm2_fv2\all-Mozambique\rlds_Amon_CESM2-FV2_historical_r1i1p1f1_gn_19500115-20141215_v20191120.nc

 The final path for nc file exists ? True



C:\Users\CLMRX\AppData\Local\Temp\1\ipykernel_916\3903570655.py:84: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  dates=ds.indexes['time'].to_datetimeindex().floor('D')
C:\Users\CLMRX\AppData\Local\Temp\1\ipykernel_916\3903570655.py:84: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  dates=ds.indexes['time'].to_datetimeindex().floor('D')
C:\Users\CLMRX\AppData\Local\Temp\1\ipykernel_916\3903570655.py:84: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar. 

path_for_file \\COWI.net\projects\A245000\A248363\CRVA\Datasets\radiation\surface_downwelling_longwave_radiation\Copernicus-CMIP6\raw_data\1950-2014\historical\cesm2_waccm\all-Mozambique does exist Function copernicus search for nc
The file is in the path Function copernicus search for nc

Before path_length, The final path for the nc file is: \\COWI.net\projects\A245000\A248363\CRVA\Datasets\radiation\surface_downwelling_longwave_radiation\Copernicus-CMIP6\raw_data\1950-2014\historical\cesm2_waccm\all-Mozambique\rlds_Amon_CESM2-WACCM_historical_r1i1p1f1_gn_19500115-20141215_v20190227.nc

 The final path for nc file exists ? True

After path_length, The final path for the nc file is: \\COWI.net\projects\A245000\A248363\CRVA\Datasets\radiation\surface_downwelling_longwave_radiation\Copernicus-CMIP6\raw_data\1950-2014\historical\cesm2_waccm\all-Mozambique\rlds_Amon_CESM2-WACCM_historical_r1i1p1f1_gn_19500115-20141215_v20190227.nc

 The final path for nc file exists ? True



...

path_for_file \\COWI.net\projects\A245000\A248363\CRVA\Datasets\radiation\surface_downwelling_longwave_radiation\Copernicus-CMIP6\raw_data\1950-2014\historical\noresm2_mm\all-Mozambique does exist Function copernicus search for nc
The file is in the path Function copernicus search for nc

Before path_length, The final path for the nc file is: \\COWI.net\projects\A245000\A248363\CRVA\Datasets\radiation\surface_downwelling_longwave_radiation\Copernicus-CMIP6\raw_data\1950-2014\historical\noresm2_mm\all-Mozambique\rlds_Amon_NorESM2-MM_historical_r1i1p1f1_gn_19500116-20141216_v20191108.nc

 The final path for nc file exists ? True

After path_length, The final path for the nc file is: \\COWI.net\projects\A245000\A248363\CRVA\Datasets\radiation\surface_downwelling_longwave_radiation\Copernicus-CMIP6\raw_data\1950-2014\historical\noresm2_mm\all-Mozambique\rlds_Amon_NorESM2-MM_historical_r1i1p1f1_gn_19500116-20141216_v20191108.nc

 The final path for nc file exists ? True



C:\Users\CLMRX\AppData\Local\Temp\1\ipykernel_916\3903570655.py:84: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  dates=ds.indexes['time'].to_datetimeindex().floor('D')
C:\Users\CLMRX\AppData\Local\Temp\1\ipykernel_916\3903570655.py:84: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  dates=ds.indexes['time'].to_datetimeindex().floor('D')
C:\Users\CLMRX\AppData\Local\Temp\1\ipykernel_916\3903570655.py:84: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar. 

path_for_file \\COWI.net\projects\A245000\A248363\CRVA\Datasets\radiation\surface_downwelling_longwave_radiation\Copernicus-CMIP6\raw_data\1950-2014\historical\sam0_unicon\all-Mozambique does exist Function copernicus search for nc
The file is in the path Function copernicus search for nc

Before path_length, The final path for the nc file is: \\COWI.net\projects\A245000\A248363\CRVA\Datasets\radiation\surface_downwelling_longwave_radiation\Copernicus-CMIP6\raw_data\1950-2014\historical\sam0_unicon\all-Mozambique\rlds_Amon_SAM0-UNICON_historical_r1i1p1f1_gn_19500116-20141216_v20190323.nc

 The final path for nc file exists ? True

After path_length, The final path for the nc file is: \\COWI.net\projects\A245000\A248363\CRVA\Datasets\radiation\surface_downwelling_longwave_radiation\Copernicus-CMIP6\raw_data\1950-2014\historical\sam0_unicon\all-Mozambique\rlds_Amon_SAM0-UNICON_historical_r1i1p1f1_gn_19500116-20141216_v20190323.nc

 The final path for nc file exists ? True



C:\Users\CLMRX\AppData\Local\Temp\1\ipykernel_916\3903570655.py:84: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  dates=ds.indexes['time'].to_datetimeindex().floor('D')
C:\Users\CLMRX\AppData\Local\Temp\1\ipykernel_916\3903570655.py:84: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  dates=ds.indexes['time'].to_datetimeindex().floor('D')
C:\Users\CLMRX\AppData\Local\Temp\1\ipykernel_916\3903570655.py:84: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar. 

csv file registered here : \\COWI.net\projects\A245000\A248363\CRVA\Datasets\radiation\surface_downwelling_longwave_radiation\Copernicus-CMIP6\csv\1950-2014\SomeProjectsInMoz_1950-2014_monthly_surface_downwelling_longwave_radiation_MJ_per_m-2_per_day-1.csv


In [18]:
df

surface_downwelling_longwave_radiation MJ_per_m-2_per_day-1
Name project  Latitude   Longitude Experiment Model       Date                                                                   
WTP_Mutua_EIB -19.319372 35.0      historical cesm2       1950-01-15                                          37.931477          
                                                          1950-02-14                                          36.754929          
                                                          1950-03-15                                          36.897514          
                                                          1950-04-15                                          35.461895          
                                                          1950-05-15                                          33.910576          
...                                                                                                                 ...          
Pemba_EIB     -12.722513 40.0      historical sam0_unicon 2014-08-16                                          31.850876          
                                                          2014-09-16                                          32.366177          
                                                          2014-10-16                                          34.060936          
                                                          2014-11-16                                          34.862659          
                                                          2014-12-16                                          35.393719          

[87360 rows x 1 columns]